In [24]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from ast import literal_eval

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_distances
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer as PS

import joblib

import pickle

from pprint import pprint


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# data = pd.read_csv('../full_dataset.csv')

In [4]:
# data.info()

In [5]:
# data = data.loc[data['source'] == 'Gathered']

In [6]:
# data.drop(columns='source', inplace=True)

In [7]:
# data.info()

In [8]:
# def literal_return(val):
#     try:
#         return literal_eval(val)
#     except (ValueError, SyntaxError) as e:
#         return val

# def df_str_to_literal(df, column_name):
#     df[column_name] = df[column_name].apply(literal_return)
#     return df[column_name]

In [9]:
# def clean_df(df, columns_list):
#     for col in columns_list:
#         df[col] = df_str_to_literal(df, col)
#     return None

## CLEANING DF

In [10]:
# clean_df(data, ['ingredients', 'directions', 'NER'])

In [11]:
# data['bag_of_words'] = ''
# columns_list = ['title', 'ingredients', 'directions']
# for col in columns_list:
#     if col == 'title':
#         data['bag_of_words'] += data[col] + ' '
#     if col == 'ingredients':
#         data['bag_of_words'] += data[col].apply(' '.join) + ' '
#     if col == 'directions':
#         data['bag_of_words'] += data[col].apply(' '.join)
# data.head()

In [14]:
# bow_data = data[['title','bag_of_words']]
# bow_data.head()

,title,bag_of_words
0,No-Bake Nut Cookies,No-Bake Nut Cookies 1 c. firmly packed brown s...
1,Jewell Ball'S Chicken,Jewell Ball'S Chicken 1 small jar chipped beef...
2,Creamy Corn,Creamy Corn 2 (16 oz.) pkg. frozen corn 1 (8 o...
3,Chicken Funny,Chicken Funny 1 large whole chicken 2 (10 1/2 ...
4,Reeses Cups(Candy),Reeses Cups(Candy) 1 c. peanut butter 3/4 c....


In [12]:
# documents = bow_data.bag_of_words
# documents

In [13]:
# def clean_document(document):
#     '''
#     Takes in a string.
#     Returns cleaned string.
#     '''
#     # lowercase the strings
#     doc_lower = document.lower() 

#     #tokenize
#     tokens = word_tokenize(doc_lower) 
    
#     # remove punctuation
#     punc = set(string.punctuation)
#     tokens_no_punc = [word for word in tokens if word not in punc]
   
#     # remove stopwords
#     s_words = set(stopwords.words('english'))
#     s_words_list = ['tablespoon', 'tbsp', 'teaspoon', 'tsp', 'cup', 'oz', 'lb', 'c.']
#     for word in s_words_list:
#         s_words.add(word)
#     tokens_no_sw = [word for word in tokens_no_punc if word not in s_words]
    
#     # stem the words to get rid of multiple forms of the same word
#     porter = PS()
#     tokens_stemmed = [porter.stem(word) for word in tokens_no_sw]
    
#     # join all words into one string
#     cleaned_doc = ' '.join(tokens_stemmed)
    
#     return cleaned_doc

## CLEANING DATA

In [14]:
# bow_data['cleaned_bow'] = bow_data['bag_of_words'].apply(clean_document)
# bow_data

In [15]:
# data.to_pickle('./models/full_data_df_pickle_4.pkl', protocol=4)

In [16]:
# data.to_pickle('./models/full_data_df_pickle_5.pkl', protocol=5)

In [17]:
# bow_data.to_pickle('./models/training_df_pickle_4.pkl', protocol=4)

In [18]:
# bow_data.to_pickle('./models/training_df_pickle_5.pkl', protocol=5)

## TRAINING

In [20]:
num_features = 1000
ngram_range=(1,3)

vec = CountVectorizer(max_df=0.85, 
                      min_df=10,
                      ngram_range=ngram_range,
                      max_features=num_features)


In [21]:
joblib.dump(vec, './models/full_vec_pickle4.joblib', protocol=4)

['./models/full_vec_pickle4.joblib']

In [22]:
joblib.dump(vec, './models/full_vec_pickle5.joblib', protocol=5)

['./models/full_vec_pickle5.joblib']

In [25]:
with open('./models/training_df_pickle_4.pkl', 'rb') as f:
    bow_data = pickle.load(f)

In [ ]:
bow_data.info()

In [26]:
docs_cleaned = bow_data['cleaned_bow']

In [27]:
tf = vec.fit_transform(docs_cleaned)

In [28]:
# tf_feature_names = vec.get_feature_names()
# tf_feature_names[:30]

In [29]:
# num_topics = [100]
# learning_method=['online']
# learning_offset = [10, 50, 90]
# doc_topic_prior = [None, 0.1, 0.9]
# topic_word_prior = [None, 0.1, 0.9]
# learning_decay = [0.5, 0.7, 0.9]
# batch_size = [64, 128]
# n_jobs= [-1]

In [30]:
# random_grid = {'n_components': num_topics,
#                'learning_method':learning_method,
#                'learning_offset': learning_offset,
#                'doc_topic_prior': doc_topic_prior,
#                'topic_word_prior': topic_word_prior,
#                'learning_decay': learning_decay,
#                'batch_size': batch_size,
#                'n_jobs': n_jobs}

# pprint(random_grid)

In [35]:
# lda = LatentDirichletAllocation()

# lda_grid = RandomizedSearchCV(estimator=lda, 
#                               param_distributions=random_grid,
#                               cv=5,
#                               n_iter=10,
#                               n_jobs=-1, 
#                               verbose=1)

In [36]:
# lda_grid.fit(tf_train)

In [31]:
tf_train, tf_test = train_test_split(tf, test_size=0.25)

In [34]:
lda = joblib.load('./models/lda_model_6_tid_pickle4.joblib')

In [33]:
lda.get_params()

{'batch_size': 32,
 'doc_topic_prior': 0.9,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 50,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 100,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': None,
 'topic_word_prior': 0.9,
 'total_samples': 1000000.0,
 'verbose': 0}

In [37]:
lda.fit(tf_train)

LatentDirichletAllocation(batch_size=32, doc_topic_prior=0.9,
                          learning_method='online', learning_offset=50,
                          n_components=100, n_jobs=-1, topic_word_prior=0.9)

In [38]:
print('Test Score:', lda.score(tf_test))

Test Score: -195067790.35154256


In [40]:
print('Train Score', lda.score(tf_train))

Train Score -584607435.4425652


In [41]:
print('Test Perplexity:', lda.perplexity(tf_test))

Test Perplexity: 398.5728608682162


In [43]:
print('Train Perplexity:', lda.perplexity(tf_train))

Train Perplexity: 393.9414662767103


In [44]:
joblib.dump(lda, './models/lda_model_full_tid_pickle4.joblib', protocol=4)

['./models/lda_model_full_tid_pickle4.joblib']

In [ ]:
# print(pd.DataFrame.from_dict(lda_grid.cv_results_))
# print('Test Score:', lda_grid.score(tf_test))
# print('Perplexity:', lda_grid.perplexity(tf_test))

In [ ]:
# best_lda_model = lda_grid.best_estimator_

In [ ]:
# joblib.dump(best_lda_model, './models/lda_model_full_tid_pickle4.joblib', protocol=4)

In [ ]:
# joblib.dump(best_lda_model, './models/lda_model_full_tid_pickle5.joblib', protocol=5)

In [ ]:
# joblib.dump(vec, './models/vec_full_tid_pickle4.joblib', protocol=4)

In [ ]:
# joblib.dump(vec, './models/vec_full_tid_pickle5.joblib', protocol=5)

In [ ]:
# print("Best Model's Params: ", lda_grid.best_params_)

In [ ]:
# print("Best Model's Log Likelihood Score: ", lda_grid.best_score_)

In [ ]:
# print("Best Model's Perplexity: ", best_lda_model.perplexity(tf))

In [45]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [46]:
num_top_words = 10
tf_feature_names = vec.get_feature_names()
display_topics(lda, tf_feature_names, num_top_words)

Topic 0:
minut 20 20 minut 15 20 18 15 20 minut full bake almost medium
Topic 1:
flour flour salt add flour flour mixtur sugar flour gradual combin flour gradual add togeth flour slowli
Topic 2:
water cold boil water dissolv cold water cup water gelatin soak add water water add
Topic 3:
boil bring pot bring boil simmer liquid salt water water boil larg pot boil add
Topic 4:
orang crush pineappl banana fruit cherri orang juic crush pineappl jello pear
Topic 5:
vinegar jar bag cucumb wash marinad qt lid seal pickl
Topic 6:
peel appl toss drizzl caramel medium cider tart quarter tender
Topic 7:
combin bowl larg bowl 25 bowl combin 25 minut 20 25 larg bowl combin in bowl stir
Topic 8:
whisk rack center complet batter clean come bowl larg egg wire
Topic 9:
fresh parsley thyme paprika chop fresh herb rosemari sprig chive fresh parsley
Topic 10:
place half anoth repeat and bottom almost bowl side rest
Topic 11:
pour hour option flavor 325 300 bake hour altern bake firm
Topic 12:
tablespoon ta

In [47]:
probs = lda.transform(tf)

In [48]:
def closest_recipes(keyword, recipes, probs, n_recipes=10):
    idx_arr = np.array(recipes.index)
    keyword_recipes = recipes[recipes.str.contains(keyword, case=False, regex=False)]
    keyword_samples = np.random.choice(keyword_recipes.index, size=50, replace=True)
    keyword_idxs = []
    for sample_idx in keyword_samples:
        keyword_idx = int(np.where(idx_arr == sample_idx)[0])
        keyword_idxs.append(keyword_idx)

    d={}
    for idx in keyword_idxs:
        sims = cosine_distances(probs[idx].reshape(1, -1), probs).argsort()[0]
        for sim in sims[1:n_recipes+1]:
            if sim not in d:
                d[sim] = 1
            else:
                d[sim] += 1
                
    d_sorted = [k for k, v in sorted(d.items(), key=lambda item: item[1])][:-n_recipes:-1]
    print(f'Top {n_recipes} recipes most closely related to {keyword}')
    return np.array(recipes)[d_sorted]

In [50]:
recipes = bow_data.title

In [51]:
closest_recipes('bean', recipes, probs)

Top 10 recipes most closely related to bean


array(['Red Beans And Rice', 'Calico Bean Pot', 'Championship Elk Chili',
       'Frijoles', 'Cowpoke Beans ',
       'Habichuelas Rojas Guisadas (Stewed Red Beans) ',
       'Vegetarian Chili (Good For Non-Spicy Lovers)',
       'Winter White Chili', 'Stewed Tomato Soup'], dtype=object)

In [52]:
recipes

0                          No-Bake Nut Cookies
1                        Jewell Ball'S Chicken
2                                  Creamy Corn
3                                Chicken Funny
4                         Reeses Cups(Candy)  
                          ...                 
1643093             Tuna 'N Egg Salad In Pitas
1643094                 Croque Monsieur Panini
1643095    Croque Monsieur With Cucumber Salad
1643096                       Baked Pork Chops
1643097            Date Filled Oatmeal Cookies
Name: title, Length: 1643098, dtype: object

In [ ]:
closest_recipes('vegan', recipes, probs)

In [ ]:
closest_recipes('pepperoni pizza', recipes, probs)

In [ ]:
closest_recipes('spaghetti', recipes, probs)

In [53]:
closest_recipes('chicken soup', recipes, probs)

Top 10 recipes most closely related to chicken soup


array(['Broccoli Soup', 'Mushroom Soup', 'Beef And Chicken Soup',
       'Best Broccoli Soup', 'Chicken Stew', 'Chicken Soup',
       'Quickie Tom Yum Soup', 'Chicken Soup', 'Vegetable Soup'],
      dtype=object)

Need to experiment with different parameters with the model, as well as the bag of words contents. First, let's see if we can identify the "best" number of topics using KMeans clustering.

In [ ]:
closest_recipes('lentil', recipes, probs)

In [ ]:
closest_recipes('cheese pizza', recipes, probs)

## KMEANS

In [54]:
vectorizer = TfidfVectorizer(max_features=1000)
docs_vec = vectorizer.fit_transform(docs_cleaned)
features = vectorizer.get_feature_names()

In [55]:
k=100
kmeans = KMeans(n_clusters=k)
kmeans.fit(docs_vec)
score = silhouette_score(docs_vec, kmeans.labels_)
print(f'k = {k}, silhouette score = {score}')

KeyboardInterrupt: 

In [ ]:
joblib.dump(kmeans, './models/kmeans_full.joblib')
joblib.dump(vectorizer, './models/tf_vec_full.joblib')


In [ ]:
kmeans = joblib.load('./models/kmeans_full.joblib')
tf_vec = joblib.load('./models/tf_vec_full.joblib')

In [ ]:
# Find the top 10 features for each cluster.
n_features = 10
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-(n_features+1):-1]
print("top features (words) for each cluster:")
for num, centroid in enumerate(top_centroids):
    print(f"{num}, {', '.join(features[i] for i in centroid)}")

In [ ]:
print("Random sample of texts in each cluster \n")
assigned_cluster = kmeans.transform(docs_vec).argmin(axis=1)
for i in range(kmeans.n_clusters):
    cluster = np.arange(0, docs_vec.shape[0])[assigned_cluster==i]
    sample_recipes = np.random.choice(cluster, 10, replace=False)
    
    print(f'\n cluster {i}:')
    for idx in sample_recipes:
        print(f'{recipes.iloc[idx]}')

In [ ]:
recipe_count = []
for i in range(kmeans.n_clusters):
    cluster = np.arange(0, docs_vec.shape[0])[assigned_cluster==i]
    recipe_count.append(len(cluster))
    print(f"Cluster {i}: {len(cluster)} recipes")

In [144]:
top_words = []
n_features = 1
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-(n_features+1):-1]
for num, centroid in enumerate(top_centroids):
    top_words.append(', '.join(features[i] for i in centroid))

In [ ]:
fig, ax = plt.subplots(figsize=(24, 8))
ax.set_title('Recipe Count By Cluster', fontsize = 24)
ax.set_ylabel('Count', fontsize = 24)
plt.yticks(fontsize = 20)
ax.set_xticklabels(top_words, fontsize = 18, rotation = 90)
ax.bar(top_words, recipe_count);